In [8]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import seaborn as sns
from skimage import io
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

In [3]:
ROOT_DIR = '/media/mark/Data/deeplesion'
# ROOT_DIR = '../Data/'
CSV_NAME = 'DL_info.csv'

In [4]:
df = pd.read_csv(os.path.join(ROOT_DIR, CSV_NAME))
df.head()

,File_name,Patient_index,Study_index,Series_ID,Key_slice_index,Measurement_coordinates,Bounding_boxes,Lesion_diameters_Pixel_,Normalized_lesion_location,Coarse_lesion_type,Possibly_noisy,Slice_range,Spacing_mm_px_,Image_size,DICOM_windows,Patient_gender,Patient_age,Train_Val_Test
0,000001_01_01_109.png,1,1,1,109,"233.537, 95.0204, 234.057, 106.977, 231.169, 1...","226.169, 90.0204, 241.252, 111.977","11.9677, 5.10387","0.44666, 0.283794, 0.434454",3,0,"103, 115","0.488281, 0.488281, 5","512, 512","-175, 275",F,62.0,3
1,000001_02_01_014.png,1,2,1,14,"224.826, 289.296, 224.016, 305.294, 222.396, 2...","217.396, 284.296, 233.978, 310.294","16.019, 6.61971","0.431015, 0.485238, 0.340745",3,0,"8, 23","0.314453, 0.314453, 5","512, 512","-175, 275",F,72.0,3
2,000001_02_01_017.png,1,2,1,17,"272.323, 320.763, 246.522, 263.371, 234.412, 3...","229.412, 258.371, 285.221, 325.763","62.9245, 48.9929","0.492691, 0.503106, 0.351754",3,0,"8, 23","0.314453, 0.314453, 5","512, 512","-175, 275",F,72.0,3
3,000001_03_01_088.png,1,3,1,88,"257.759, 157.618, 260.018, 133.524, 251.735, 1...","246.735, 128.524, 270.288, 162.618","24.1998, 13.6123","0.498999, 0.278924, 0.452792",3,0,"58, 118","0.732422, 0.732422, 1","512, 512","-175, 275",F,73.0,3
4,000001_04_01_017.png,1,4,1,17,"304.019, 230.585, 292.217, 211.789, 304.456, 2...","287.217, 206.789, 309.456, 235.585","22.1937, 9.8065","0.572678, 0.42336, 0.445674",3,0,"11, 23","0.666016, 0.666016, 5","512, 512","-175, 275",F,73.0,3


In [6]:
labeled = df.loc[df["Train_Val_Test"] > 1]
print('{} labled images from {} patients'.format(labeled["Patient_index"].count(), labeled["Patient_index"].nunique()))

9816 labled images from 1368 patients


For Initial Exploration I am planning on using 32 random images for each class and 4 test images and validation images for each class
Depending on thes results I will do a more formal 60 / 20 / 20 split

In [45]:
train = []
test = []
validation = []

type_indexes = {x: None for x in range(1,9)}

for i in range(1, 9):
    sample = labeled.loc[labeled["Coarse_lesion_type"] == i].sample(40, random_state=123454321).index.values
    train.extend(sample[:32])
    test.extend(sample[32:36])
    validation.extend(sample[36:])
print(len(set(train)), len(set(test)), len(set(validation)))


256 32 32


In [47]:
df.iloc[train].to_csv('../Classification/train_set.tsv', sep='\t', index=False)
df.iloc[test].to_csv('../Classification/test_set.tsv', sep='\t', index=False)
df.iloc[validation].to_csv('../Classification/validation_set.tsv', sep='\t', index=False)

In [7]:
train = []
test = []
validation = []

type_indexes = {x: None for x in range(1,9)}



for i in range(1, 9):
    type_indexes[i] =  labeled.loc[labeled["Coarse_lesion_type"] == i].index.values
    
# print(len(set(train)), len(set(test)), len(set(validation)))
print(type_indexes)

{1: array([   41,    72,    73,  1041,  1822,  1824,  1827,  1964,  1965,
        1966,  1967,  1968,  1969,  2541,  2545,  3087,  3095,  3096,
        3097,  3165,  3237,  3240,  3242,  3247,  3248,  3251,  3716,
        4174,  4207,  4208,  4212,  4213,  4214,  4215,  4216,  4217,
        4220,  4222,  4229,  4230,  4231,  4304,  4309,  4915,  5351,
        5878,  6003,  7343,  7344,  7345,  8539,  8685,  9067, 10262,
       10265, 10808, 11177, 11971, 12134, 12379, 12396, 12397, 12398,
       13035, 13036, 13037, 13089, 14034, 14037, 14039, 14388, 14393,
       14394, 15077, 15080, 15081, 15082, 15083, 15084, 15087, 15088,
       15089, 15090, 15091, 15387, 15388, 15958, 15959, 16082, 16503,
       16635, 16636, 16992, 16995, 17995, 17996, 18580, 18796, 18898,
       18899, 18900, 18911, 18912, 18913, 18914, 18937, 18938, 18939,
       18940, 18942, 19277, 19279, 19280, 19281, 19282, 19283, 19287,
       19292, 19293, 19584, 19585, 19586, 19587, 19588, 19589, 19590,
       19880, 19

In [16]:
train = []
test = []
validation = []

for i in range(1, 9):
    X_train, X_test = train_test_split(
        type_indexes[i], test_size=0.4, random_state=42)
    X_val, X_test = train_test_split(
        X_test, test_size=0.5, random_state=42)
    train.extend(X_train)
    test.extend(X_test)
    validation.extend(X_val)
print(len(train), len(test), len(validation))
print(set(train) & set(test) & set(validation))

5887 1967 1962
set()


In [17]:
df.iloc[train].to_csv('../Classification/train_set.tsv', sep='\t', index=False)
df.iloc[test].to_csv('../Classification/test_set.tsv', sep='\t', index=False)
df.iloc[validation].to_csv('../Classification/validation_set.tsv', sep='\t', index=False)